In [1]:
%matplotlib inline

import pandas as pd

import numpy  as np
from matplotlib import pyplot as plt

In [2]:
LGAs = pd.read_csv("LGA_Disadvantage_index_2011.csv", index_col = "LGA", usecols = ["LGA", "Population_2011", "Overall_quintile"])

In [3]:
LGAs.head(5)

,Population_2011,Overall_quintile
LGA,,
Aurukun,"1,289",1.0
Balonne,"4,719",1.0
Banana,"14,440",2.0
Barcaldine,"3,217",2.0
Barcoo,352,1.0


In [4]:
Tenancies = pd.read_csv("Socialhousingtenancies_2016.csv", usecols = ["LGA", "NumberBedrooms", "MarketRent", "Occupancy"])
Tenancies["MarketRent"] = Tenancies["MarketRent"].replace("\$","",regex = True).astype(float)

In [5]:
Tenancies.head()

,NumberBedrooms,MarketRent,Occupancy,LGA
0,3,255.0,Under-occupied,Ipswich
1,3,375.0,Under-occupied,Brisbane
2,3,370.0,Not overcrowded or under-occupied,Brisbane
3,3,375.0,Under-occupied,Brisbane
4,4,420.0,Under-occupied,Brisbane


In [6]:
Properties = Tenancies[["LGA", "NumberBedrooms"]].groupby("LGA").count()
Properties.rename(columns = {"NumberBedrooms" : "# of properties"}, inplace = True)



In [7]:
Properties.head(10)

,# of properties
LGA,
Balonne,91
Banana,227
Barcaldine,19
Barcoo,5
Blackall Tambo,11
Boulia,22
Brisbane,15531
Bundaberg,995
Burdekin,154


In [8]:
test = LGAs.join(Properties, how = "inner")
test.head()

,Population_2011,Overall_quintile,# of properties
LGA,,,
Balonne,"4,719",1.0,91
Banana,"14,440",2.0,227
Barcaldine,"3,217",2.0,19
Barcoo,352,1.0,5
Blackall Tambo,"2,209",1.0,11


In [9]:
onebdr = Tenancies[Tenancies["NumberBedrooms"]==1][["LGA", "NumberBedrooms"]].groupby("LGA").count()
onebdr.rename(columns = {"NumberBedrooms" : "# of 1bdrs"}, inplace = True)
onebdr.head()


,# of 1bdrs
LGA,
Balonne,3
Banana,9
Barcaldine,4
Brisbane,5475
Bundaberg,234


In [10]:
twobdr = Tenancies[Tenancies["NumberBedrooms"]==2][["LGA", "NumberBedrooms"]].groupby("LGA").count()
twobdr.rename(columns = {"NumberBedrooms" : "# of 2bdrs"}, inplace = True)
twobdr.head()


,# of 2bdrs
LGA,
Balonne,27
Banana,30
Barcaldine,3
Barcoo,3
Blackall Tambo,5


In [11]:
Threebdr = Tenancies[Tenancies["NumberBedrooms"]>2][["LGA", "NumberBedrooms"]].groupby("LGA").count()
Threebdr.rename(columns = {"NumberBedrooms" : "# of 3+bdrs"}, inplace = True)
Threebdr.head()

,# of 3+bdrs
LGA,
Balonne,61
Banana,188
Barcaldine,12
Barcoo,2
Blackall Tambo,6


In [12]:
bdtypes = pd.concat([onebdr, twobdr, Threebdr], axis = 1)
bdtypes = bdtypes.fillna(0)
bdtypes.head()


,# of 1bdrs,# of 2bdrs,# of 3+bdrs
Balonne,3.0,27,61
Banana,9.0,30,188
Barcaldine,4.0,3,12
Barcoo,0.0,3,2
Blackall Tambo,0.0,5,6


In [13]:
prbdrs = bdtypes.divide(bdtypes.sum(axis=1), axis=0)
prbdrs.rename(columns = {"# of 3+bdrs" : "% of 3+bdrs", "# of 2bdrs" : "% of 2bdrs", "# of 1bdrs" : "% of 1bdrs"}, inplace = True)
prbdrs["% of 1bdrs"] = pd.Series(["{0:.1f}%".format(val * 100) for val in prbdrs["% of 1bdrs"]], index = prbdrs.index)
prbdrs["% of 2bdrs"] = pd.Series(["{0:.1f}%".format(val * 100) for val in prbdrs["% of 2bdrs"]], index = prbdrs.index)
prbdrs["% of 3+bdrs"] = pd.Series(["{0:.1f}%".format(val * 100) for val in prbdrs["% of 3+bdrs"]], index = prbdrs.index)
prbdrs.head()

,% of 1bdrs,% of 2bdrs,% of 3+bdrs
Balonne,3.3%,29.7%,67.0%
Banana,4.0%,13.2%,82.8%
Barcaldine,21.1%,15.8%,63.2%
Barcoo,0.0%,60.0%,40.0%
Blackall Tambo,0.0%,45.5%,54.5%


In [14]:
Occupancy = Tenancies[["LGA", "Occupancy"]].replace("Not overcrowded or under-occupied",0, regex = True).replace("Under-occupied",1, regex = True).replace("Overcrowded",2, regex = True)
Occupancy.head()

,LGA,Occupancy
0,Ipswich,1
1,Brisbane,1
2,Brisbane,0
3,Brisbane,1
4,Brisbane,1


In [15]:
underocc = Occupancy[Occupancy["Occupancy"]==1].groupby("LGA").count()
underocc.rename(columns = {"Occupancy" : "# underocc"}, inplace = True)
underocc.head()

,# underocc
LGA,
Balonne,14
Banana,75
Barcaldine,5
Barcoo,2
Blackall Tambo,3


In [16]:
overocc = Occupancy[Occupancy["Occupancy"]==2].groupby("LGA").count()
overocc.rename(columns = {"Occupancy" : "# overocc"}, inplace = True)
overocc.head()

,# overocc
LGA,
Balonne,6
Banana,7
Boulia,1
Brisbane,658
Bundaberg,40


In [17]:
normocc = Occupancy[Occupancy["Occupancy"]==0].groupby("LGA").count()
normocc.rename(columns = {"Occupancy" : "# normalocc"}, inplace = True)
normocc.head()

,# normalocc
LGA,
Balonne,71
Banana,145
Barcaldine,14
Barcoo,3
Blackall Tambo,8


In [18]:
occtype = pd.concat([underocc, overocc, normocc], axis = 1)
occtype = occtype.fillna(0)
occtype.head()

,# underocc,# overocc,# normalocc
Balonne,14.0,6.0,71
Banana,75.0,7.0,145
Barcaldine,5.0,0.0,14
Barcoo,2.0,0.0,3
Blackall Tambo,3.0,0.0,8


In [19]:
procc = occtype.divide(occtype.sum(axis=1), axis=0)
procc.rename(columns = {"# underocc" : "% underocc", "# overocc" : "% overocc", "# normalocc" : "# normalocc"}, inplace = True)
procc = procc[["% underocc", "% overocc"]]
procc["% underocc"] = pd.Series(["{0:.1f}%".format(val * 100) for val in procc["% underocc"]], index = procc.index)
procc["% overocc"] = pd.Series(["{0:.1f}%".format(val * 100) for val in procc["% overocc"]], index = procc.index)
procc.head()

,% underocc,% overocc
Balonne,15.4%,6.6%
Banana,33.0%,3.1%
Barcaldine,26.3%,0.0%
Barcoo,40.0%,0.0%
Blackall Tambo,27.3%,0.0%


In [20]:
avgrent = Tenancies.groupby("LGA")[["MarketRent"]].mean()
avgrent = avgrent.round(0)
avgrent.head(10)

,MarketRent
LGA,
Balonne,172.0
Banana,235.0
Barcaldine,138.0
Barcoo,81.0
Blackall Tambo,110.0
Boulia,144.0
Brisbane,306.0
Bundaberg,233.0
Burdekin,229.0


In [21]:
Maintenance = pd.read_csv("Socialhousingmaintenance_2016.csv", usecols = ["LGA_Name", "Cost"])
Maintenance["Cost"] = Maintenance["Cost"].replace("\$","",regex = True).replace("\,","",regex = True).astype(float)
Maintenance.rename(columns = {"LGA_Name" : "LGA"}, inplace = True)
Maintenance.rename(columns = {"Cost" : "Average Maintenance $"}, inplace = True)
Maintenance.head()

,Average Maintenance $,LGA
0,243.14,Brisbane
1,344.25,Brisbane
2,838.58,Brisbane
3,2706.03,Brisbane
4,441.46,Brisbane


In [22]:
avgmain = Maintenance[["LGA", "Average Maintenance $"]].groupby(Maintenance["LGA"].str.title()).mean()
avgmain.head(10)

,Average Maintenance $
LGA,
Aurukun,21896.340854
Balonne,2640.401709
Banana,3991.989788
Barcaldine,2925.275923
Barcoo,854.387727
Blackall Tambo,1320.567377
Boulia,2949.720407
Brisbane,2415.287109
Bundaberg,2567.863069


In [23]:
housing = LGAs.join(Properties, how = "inner")
housing = pd.concat([housing, prbdrs, avgrent, procc], axis = 1)
housing = housing.join(avgmain, how = "inner")
housing = housing.round(1)
housing


,Population_2011,Overall_quintile,# of properties,% of 1bdrs,% of 2bdrs,% of 3+bdrs,MarketRent,% underocc,% overocc,Average Maintenance $
Balonne,"4,719",1.0,91.0,3.3%,29.7%,67.0%,172.0,15.4%,6.6%,2640.4
Banana,"14,440",2.0,227.0,4.0%,13.2%,82.8%,235.0,33.0%,3.1%,3992.0
Barcaldine,"3,217",2.0,19.0,21.1%,15.8%,63.2%,138.0,26.3%,0.0%,2925.3
Barcoo,352,1.0,5.0,0.0%,60.0%,40.0%,81.0,40.0%,0.0%,854.4
Blackall Tambo,"2,209",1.0,11.0,0.0%,45.5%,54.5%,110.0,27.3%,0.0%,1320.6
Boulia,478,1.0,22.0,0.0%,36.4%,63.6%,144.0,9.1%,4.5%,2949.7
Brisbane,"1,033,614",5.0,15531.0,35.3%,28.8%,36.0%,306.0,12.1%,4.2%,2415.3
Bundaberg,"89,762",1.0,995.0,23.5%,29.0%,47.4%,233.0,14.4%,4.0%,2567.9
Burdekin,"17,373",1.0,154.0,12.3%,21.4%,66.2%,229.0,20.1%,7.1%,2591.8
Burke,521,1.0,12.0,0.0%,25.0%,75.0%,142.0,25.0%,33.3%,8721.0


In [24]:
housing.to_csv('housing.csv')

In [43]:
lga = 'Banana'
housing.loc[lga].to_frame().to_html()
#housing.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Banana</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Population_2011</th>\n      <td>14,440</td>\n    </tr>\n    <tr>\n      <th>Overall_quintile</th>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th># of properties</th>\n      <td>227</td>\n    </tr>\n    <tr>\n      <th>% of 1bdrs</th>\n      <td>4.0%</td>\n    </tr>\n    <tr>\n      <th>% of 2bdrs</th>\n      <td>13.2%</td>\n    </tr>\n    <tr>\n      <th>% of 3+bdrs</th>\n      <td>82.8%</td>\n    </tr>\n    <tr>\n      <th>MarketRent</th>\n      <td>235</td>\n    </tr>\n    <tr>\n      <th>% underocc</th>\n      <td>33.0%</td>\n    </tr>\n    <tr>\n      <th>% overocc</th>\n      <td>3.1%</td>\n    </tr>\n    <tr>\n      <th>Average Maintenance $</th>\n      <td>3992</td>\n    </tr>\n  </tbody>\n</table>'